In [1]:
from smolagents import tool 
# import packages that are used in our tools
import requests
from bs4 import BeautifulSoup
import json

@tool
def get_hugging_face_top_daily_paper() -> str:
    """
    This is a tool that returns the most upvoted paper on Hugging Face daily papers.
    It returns the title of the paper
    """
    try:
      url = "https://huggingface.co/papers"
      response = requests.get(url)
      response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
      soup = BeautifulSoup(response.content, "html.parser")

      # Extract the title element from the JSON-like data in the "data-props" attribute
      containers = soup.find_all('div', class_='SVELTE_HYDRATER contents')
      top_paper = ""

      for container in containers:
          data_props = container.get('data-props', '')
          if data_props:
              try:
                  # Parse the JSON-like string
                  json_data = json.loads(data_props.replace('&quot;', '"'))
                  if 'dailyPapers' in json_data:
                      top_paper = json_data['dailyPapers'][0]['title']
              except json.JSONDecodeError:
                  continue

      return top_paper
    except requests.exceptions.RequestException as e:
      print(f"Error occurred while fetching the HTML: {e}")
      return None

In [2]:
from huggingface_hub import HfApi

@tool
def get_paper_id_by_title(title: str) -> str:
    """
    This is a tool that returns the arxiv paper id by its title.
    It returns the title of the paper

    Args:
        title: The paper title for which to get the id.
    """
    api = HfApi()
    papers = api.list_papers(query=title)
    if papers:
        paper = next(iter(papers))
        return paper.id
    else:
        return None

In [3]:
import arxiv

@tool
def download_paper_by_id(paper_id: str) -> None:
    """
    This tool gets the id of a paper and downloads it from arxiv. It saves the paper locally 
    in the current directory as "paper.pdf".

    Args:
        paper_id: The id of the paper to download.
    """
    paper = next(arxiv.Client().results(arxiv.Search(id_list=[paper_id])))
    paper.download_pdf(filename="paper.pdf")
    return None

In [4]:
from pypdf import PdfReader

@tool
def read_pdf_file(file_path: str) -> str:
    """
    This function reads the first three pages of a PDF file and returns its content as a string.
    Args:
        file_path: The path to the PDF file.
    Returns:
        A string containing the content of the PDF file.
    """
    content = ""
    reader = PdfReader('paper.pdf')
    print(len(reader.pages))
    pages = reader.pages[:3]
    for page in pages:
        content += page.extract_text()
    return content

In [ ]:
from smolagents import CodeAgent, ToolCallingAgent
from smolagents import InferenceClientModel, LiteLLMModel
import os
from dotenv import load_dotenv
load_dotenv()

# Override default api endpoint URL for DashScope in LiteLLMModel if your api key
# is not -intl account.
# DASHSCOPE_API_BASE=https://dashscope.aliyuncs.com/compatible-mode/v1
# DASHSCOPE_API_KEY=<your_api_key>

model = LiteLLMModel(model_id="dashscope/qwen-turbo")

agent = CodeAgent(tools=[get_hugging_face_top_daily_paper,
                         get_paper_id_by_title,
                         download_paper_by_id,
                         read_pdf_file],
                  model=model,
                  max_steps=10)

agent.run(
    "Summarize today's top paper on Hugging Face daily papers by reading it.",
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Summarize today's top paper on Hugging Face daily papers by reading it.                                         │
│                                                                                                                 │
╰─ LiteLLMModel - dashscope/qwen-turbo ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  top_paper_title = get_hugging_face_top_daily_paper()                                                             
  print(f"Top paper title: {top_paper_title}")                                                                     
                                                                                                                   
  paper_id = get_paper_id_by_title(title=top_paper_title)                                                          
  print(f"Paper ID: {paper_id}")                                                                                   
                                                                                                                   
  download_paper_by_id(paper_id=paper_id)                                                                          
  print("Paper downloaded.")                                                                                       
                                                                                                                   
  paper_content = read_pdf_file(file_path="paper.pdf")                                                             
  print(f"Paper content: {paper_content}")                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

26


Execution logs:
Top paper title: MMR1: Enhancing Multimodal Reasoning with Variance-Aware Sampling and
  Open Resources
Paper ID: 2509.21268
Paper downloaded.
Paper content: Preprint. Work in progress.
MMR1: ENHANCINGMULTIMODALREASONING WITH
VARIANCE-AWARESAMPLING ANDOPENRESOURCES
Sicong Leng1,2,∗ Jing Wang1,∗ Jiaxi Li3,∗ Hao Zhang2,∗ Zhiqiang Hu2
Boqiang Zhang2 Yuming Jiang2 Hang Zhang2 Xin Li2 Lidong Bing2
Deli Zhao2 Wei Lu1 Yu Rong2 Aixin Sun1,† Shijian Lu1,†
1Nanyang Technological University
2DAMO Academy, Alibaba Group
3Singapore University of Technology and Design
∗Equal Contributions †Correspondence
ABSTRACT
Large multimodal reasoning models have achieved rapid progress, but their ad-
vancement is constrained by two major limitations: the absence of open, large-scale,
high-quality long chain-of-thought (CoT) data, and the instability of reinforcement
learning (RL) algorithms in post-training. Group Relative Policy Optimization
(GRPO), the standard framework for RL fine-tuning, is prone togradient vanishing
when reward variance is low, which weakens optimization signals and impairs con-
vergence. This work makes three contributions: (1) We propose Variance-Aware
Sampling (V AS), a data selection strategy guided by Variance Promotion Score
(VPS) that combines outcome variance and trajectory diversity to promote reward
variance and stabilize policy optimization. (2) We release large-scale, carefully
curated resources containing ∼1.6M long CoT cold-start data and ∼15k RL QA
pairs, designed to ensure quality, difficulty, and diversity, along with a fully repro-
ducible end-to-end training codebase. (3) We open-source a family of multimodal
reasoning models in multiple scales, establishing standardized baselines for the
community. Experiments across mathematical reasoning benchmarks demonstrate
the effectiveness of both the curated data and the proposed V AS. Comprehensive
ablation studies and analyses provide further insight into the contributions of each
component. In addition, we theoretically establish that reward variance lower-
bounds the expected policy gradient magnitude, with V AS serving as a practical
mechanism to realize this guarantee. Our code, data, and checkpoints are available
athttps://github.com/LengSicong/MMR1.
1 INTRODUCTION
Recent advances in large language and multimodal reasoning models have markedly improved
performance on complex tasks such as mathematics, science, and open-domain problem solving.
Reinforcement learning (RL) plays a central role in these developments by optimizing models with
process- or outcome-based rewards (Lightman et al., 2024; Wang et al., 2024c; Li et al., 2025b).
Group Relative Policy Optimization (GRPO; Shao et al. (2024)) has emerged as a widely adopted RL
framework due to its efficiency and scalability, and has been successfully applied to both language
models (Guo et al., 2025) and multimodal models (Meng et al., 2025a; Wang et al., 2025b; Tan et al.,
2025; Leng et al., 2025). However, GRPO is inherently susceptible togradient vanishing: when
sampled rewards have low variance, relative advantages collapse toward zero, weakening optimization
signals and destabilizing training (Razin et al., 2024; 2025). This issue persists across both unimodal
and multimodal contexts, posing a fundamental challenge to effective RL optimization.
In parallel, the progress of multimodal reasoning research is influenced by the limited availability of
open, large-scale, high-quality long chain-of-thought (CoT) data. Compared with text-only reasoning,
where multiple datasets are publicly accessible (Guha et al., 2025; Muennighoff et al., 2025), multi-
modal training often relies on more restricted resources, which may constrain reproducibility and
1
arXiv:2509.21268v1  [cs.CV]  25 Sep 2025Preprint. Work in progress.
training data
random sampler
weighted sampler
trainingbatch mix
policy model
rolloutsrewards calculation
update policysync policy & re-calculate weights
policy model
rolloutsVPS calc

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  summary = "The paper introduces Variance-Aware Sampling (VAS), a data selection strategy to mitigate gradient    
  vanishing in GRPO-based training for multimodal reasoning models. It proposes the Variance Promotion Score       
  (VPS), which combines Outcome Variance Score (OVS) and Trajectory Diversity Score (TDS) to promote reward        
  variance and stabilize policy optimization. The paper also releases large-scale datasets for supervised          
  fine-tuning and RL, along with a reproducible codebase and open models."                                         
  final_answer(summary)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: The paper introduces Variance-Aware Sampling (VAS), a data selection strategy to mitigate gradient 
vanishing in GRPO-based training for multimodal reasoning models. It proposes the Variance Promotion Score (VPS), 
which combines Outcome Variance Score (OVS) and Trajectory Diversity Score (TDS) to promote reward variance and 
stabilize policy optimization. The paper also releases large-scale datasets for supervised fine-tuning and RL, 
along with a reproducible codebase and open models.

[Step 2: Duration 4.19 seconds| Input tokens: 7,619 | Output tokens: 275]

'The paper introduces Variance-Aware Sampling (VAS), a data selection strategy to mitigate gradient vanishing in GRPO-based training for multimodal reasoning models. It proposes the Variance Promotion Score (VPS), which combines Outcome Variance Score (OVS) and Trajectory Diversity Score (TDS) to promote reward variance and stabilize policy optimization. The paper also releases large-scale datasets for supervised fine-tuning and RL, along with a reproducible codebase and open models.'